In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
x = np.linspace(0,50,501)
y = np.sin(x)

In [ ]:
plt.figure(figsize=(14,6))
sns.lineplot(x=x, y=y)
plt.title('sinewave')
plt.show()

In [ ]:
df = pd.DataFrame(data=y, index=x, columns=['sine'])
test_percent = 0.1

In [ ]:
test_number = round(len(df)*test_percent)
train_point = len(df)-test_number

In [ ]:
train = df.iloc[:train_point]
test = df.iloc[train_point:]

In [ ]:
len(train), len(test)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
scaler.fit(train)

In [ ]:
train_scale = scaler.transform(train)
test_scale = scaler.transform(test)

In [ ]:
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [ ]:
length = 50
batch_size = 1
generator = TimeseriesGenerator(data=train_scale, 
                                targets=train_scale,
                                length=length,
                                batch_size=batch_size)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN, LSTM

In [ ]:
n_feature = 1

In [ ]:
model = Sequential()
model.add(SimpleRNN(50, input_shape=(length, n_feature)))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

In [ ]:
# help(SimpleRNN)

In [ ]:
model.summary()

In [ ]:
model.fit_generator(generator, epochs=5)

In [ ]:
pd.DataFrame(model.history.history).plot()
plt.show()

In [ ]:
first_eval_batch = train_scale[-length:]
first_eval_batch = first_eval_batch.reshape((1, length, n_feature))

In [ ]:
model.predict(first_eval_batch), test_scale[0]

In [ ]:
test_prediction = []

first_eval_batch = train_scale[-length:]
current_batch = first_eval_batch.reshape((1, length, n_feature))

for i in range(len(test)):
    current_prediction = model.predict(current_batch)[0]
    test_prediction.append(current_prediction)
    current_batch = np.append(current_batch[:,1:,:],
                              [[current_prediction]],
                              axis=1)

In [ ]:
true_prediction = scaler.inverse_transform(test_prediction)

In [ ]:
test['simple prediction'] = true_prediction

In [ ]:
test.plot(figsize=(16,4))
plt.show()

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=2)

In [ ]:
length = 49
batch_size = 1
generator = TimeseriesGenerator(data=train_scale, 
                                targets=train_scale,
                                length=length,
                                batch_size=batch_size)

validation = TimeseriesGenerator(data=test_scale,
                                 targets=test_scale,
                                 length=length,
                                 batch_size=batch_size)

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(length, n_feature)))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

In [ ]:
model.fit_generator(generator=generator,
                    epochs=20,
                    validation_data=validation,
                    callbacks=[early_stop])

In [ ]:
pd.DataFrame(model.history.history).plot()
plt.show()

In [ ]:
test_prediction = []

first_eval_batch = train_scale[-length:]
current_batch = first_eval_batch.reshape((1, length, n_feature))

for i in range(len(test)):
    current_prediction = model.predict(current_batch)[0]
    test_prediction.append(current_prediction)
    current_batch = np.append(current_batch[:,1:,:],
                              [[current_prediction]],
                              axis=1)

In [ ]:
true_prediction = scaler.inverse_transform(test_prediction)
test['LSTM prediction'] = true_prediction
test.plot(figsize=(16,4))
plt.show()

In [ ]:
full_scaler = MinMaxScaler()
full_scaler.fit(df)
data_scale = full_scaler.transform(df)

In [ ]:
generator = TimeseriesGenerator(data_scale,
                                data_scale,
                                length=length,
                                batch_size=1)

In [ ]:
model = Sequential()
model.add(LSTM(50, input_shape=(length, n_feature)))
model.add(Dense(1))
model.compile(loss='mse', optimizer='adam')

In [ ]:
model.fit_generator(generator=generator,
          epochs=6)

In [ ]:
forecast = []

first_eval_batch = data_scale[-length:]
current_batch = first_eval_batch.reshape((1, length, n_feature))

prediction_length = range(50)

for i in prediction_range:
    current_prediction = model.predict(current_batch)[0]
    forecast.append(current_prediction)
    current_batch = np.append(current_batch[:,1:,:],
                              [[current_prediction]],
                              axis=1)

In [ ]:
forecast = full_scaler.inverse_transform(forecast)

In [ ]:
step = 0.1
forecast_start = 50.1
forecast_end = (prediction_length*step)+forecast_start

In [ ]:
forecast_index = np.arange(start=forecast_start,
                           stop=forecast_end,
                           step=step)

In [ ]:
forecast_df = pd.DataFrame(data=forecast, index=forecast_index, columns=['sine'])


In [ ]:
plt.figure(figsize=(16,4))
plt.plot(df.index, df['sine'])
plt.plot(forecast_df.index, forecast_df['sine'])
plt.axvline(x=50, color='red')
plt.title('sine')
plt.show()